In [7]:
from pymorphy2 import MorphAnalyzer
from gensim.models import KeyedVectors
from time import time
import numpy as np
import pandas as pd

load model

In [2]:
model = KeyedVectors.load_word2vec_format('model.bin', binary=True)

In [5]:
def get_average_vector(words):
    words = [word for word in words if word in model]
    return model[words].mean(axis=0)
print(get_average_vector(['привет', 'как', 'дела']))

[-0.1267     -0.51209736 -0.16690832 -0.29150435  0.35674834  0.20734334
  0.158235   -0.03373267 -0.23175633  0.3742043   0.17194633  0.48205498
 -0.09930066 -0.151058    0.031867    0.323783    0.36028     0.14745367
  0.38986835  0.311489   -0.33128902  0.46635333  0.31798467 -0.24226
  0.12861234  0.00921533  0.0169      0.13487367 -0.02352501 -0.26025465
  0.134863   -0.09892233 -0.27922466  0.00384232 -0.08300433  0.00703567
 -0.00516333 -0.12743498  0.07026633 -0.42698732 -0.23669136  0.29773033
  0.3714107  -0.09921966 -0.11576834  0.06041167  0.21021068 -0.00220767
  0.15334734 -0.049027    0.195885   -0.09230334 -0.092242    0.23277535
  0.15144134 -0.09071799  0.03817899  0.31755933 -0.00816533 -0.3383223
 -0.02359167  0.07261167 -0.05508    -0.11849733 -0.065199    0.13171966
 -0.01821033 -0.14975631  0.72882366 -0.05273466  0.5196673   0.07776701
 -0.00391134 -0.215815    0.08391434  0.027577   -0.35810634  0.067714
  0.14507666 -0.24992865  0.003021   -0.045989    0.14582

AttributeError: module 'sklearn' has no attribute 'metrics'